In [1]:
import sys
# sys.path.append('C:/Users/xxjordan/Desktop/okex-python-sdk-api')
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as pltoff
import plotly
import talib
from abupy import abu, ml, nd, tl, pd_resample, AbuML, AbuMLPd, AbuMetricsBase 
from abupy import ABuSymbolPd, ABuScalerUtil, get_price, ABuMarketDrawing, ABuKLUtil
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import sys
from opendatatools import coin as coin_data
# import get_data
import pickle
import peakutils.peak
import numpy as np
from talib.abstract import *
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from sklearn.ensemble import IsolationForest
from plotly import tools
# from save_historical_data import get_historical_klines
# from binance.client import Client
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import tushare as ts
import baostock as bs
from sklearn.metrics import accuracy_score
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [43]:
import ccxt
bitfinex = ccxt.bitfinex2()
since = bitfinex.milliseconds() - 864000000
klines = bitfinex.fetch_ohlcv('BTC/USD',timeframe='15m',since=None, limit=5000)
df = pd.DataFrame(klines)
df.rename({0:'timestamp',1:'open',2:'high',3:'low',4:'close',5:'volume'},axis=1,inplace=True)
df['datetime'] = pd.to_datetime(df['timestamp'],unit='ms')
coin = df[['datetime','open','high','low','close','volume']]


In [16]:
# coin = pd.read_csv('bfx_btcusd.csv',names=['timestamp','open','high','low','close','volume'])
# coin['datetime'] = pd.to_datetime(coin['timestamp'],unit='ms')

# coin = coin[['datetime','open','high','low','close','volume']]

coin = pd.read_hdf('bfx_btc_1min_2017.hdf5')


# coin['rate'] = (coin['close'] - coin['close'].shift())/ coin['close'].shift()
# coin['up'] = 0
# coin['down'] = 0
# coin.loc[coin['rate']>0.005, 'up'] = 1
# coin.loc[coin['rate']<-0.005, 'down'] = 1
# coin = coin[['datetime','open','high','low','close','volume','up','down']]

coin = coin.resample(rule='30T', on='datetime', label='left', closed='left').agg(
    {'open': 'first',
     'high': 'max',
     'low': 'min',
     'close': 'last',
     'volume': 'sum',
#      'up':'sum',
#      'down':'sum'
     })
coin['datetime'] = coin.index
coin.dropna(inplace=True)
# coin = coin[-5000:]

In [2]:
coin = pd.read_hdf('bfx_btc_1min_2017.hdf5')
# coin = pd.read_hdf('bfx_btc_1min.hdf5')
threshold = 10000000
# coin['volume_sum'] = coin['volume'].cumsum()
# coin['k_line'] = coin['volume_sum']//threshold
coin['amount'] = coin['volume'] * coin['close'] # 这里的成交额只能粗略的计算一下
coin['amount_sum'] = coin['amount'].cumsum()
coin['k_line'] = coin['amount_sum']//threshold
def f1(x):
    x.reset_index(drop=True,inplace=True)
#     datetime = x.at[0,'datetime']
    datetime = x.at[0,'k_line']
    open = x.at[0,'open']
    close = x.iloc[-1]['close']
    high = x['high'].max()
    low = x['low'].min()
    volume = x['volume'].sum()
    k_line_list.append({'datetime':datetime,'open':open,'high':high,'low':low,'close':close,'volume':volume})
k_line_list=[]
coin.groupby('k_line').apply(f1)
coin = pd.DataFrame(k_line_list)
# coin = coin[-5000:]

In [3]:
def find_peak(coin,min_dist,column,rate):
    threshold = 0.02
    min_dist = min_dist
    cb = coin['close']

    highs = peakutils.peak.indexes(
        np.array(cb),
        thres=threshold/max(cb), min_dist=min_dist
    )

    cbInverted = cb * -1
    lows = peakutils.peak.indexes(
        np.array(cbInverted),
        thres=threshold/max(cbInverted), min_dist=min_dist
    )
    h = [-1 if i in highs else 0 for i in range(len(coin))]
    l = [1 if i in lows else 0 for i in range(len(coin))]
    coin['feature_sign'] = np.array(h) + np.array(l)
    coin.loc[coin['feature_sign']==0,'feature_sign'] = np.nan
    coin['position'] = coin['feature_sign'].shift()
    coin['position'].fillna(method='ffill',inplace=True)
    coin.loc[(coin['feature_sign']==1)|(coin['feature_sign']==-1),'s'] = coin['datetime']
    coin['s'] = coin['s'].shift()
    coin['s'].fillna(method='ffill',inplace=True)
    g = coin.groupby(by='s').apply(lambda x: (x.iloc[0]['open'] - x.iloc[-1]['close'])/x.iloc[0]['open'])
    g = g.reset_index(level=[0])
    coin = coin.merge(g,how='left',on='s')
    coin.rename(columns={0:'rate'}, inplace = True)
    # coin.loc[(coin['rate']>0.05),'pos'] = 2
    # coin.loc[(coin['rate']<-0.05),'pos'] = 1
    # coin['pos'].fillna(value=0,inplace=True)

    # coin.loc[(coin['rate']>-0.01)&(coin['rate']<0.01),'pos'] = 3
    # coin.loc[(coin['rate']>=0.01)&(coin['rate']<0.05),'pos'] = 1
    # coin.loc[(coin['rate']>=0.05)&(coin['rate']<0.1),'pos'] = 2
    # coin.loc[(coin['rate']>=0.1),'pos'] = 0
    # coin.loc[(coin['rate']<=-0.01)&(coin['rate']>-0.05),'pos'] = 4
    # coin.loc[(coin['rate']<=-0.05)&(coin['rate']>-0.1),'pos'] = 5
    # coin.loc[(coin['rate']<=-0.1),'pos'] = 6

    coin[column] = 0
    coin.loc[(coin['rate']>=rate),column] = -1
    coin.loc[(coin['rate']<=-rate),column] = 1

    coin.drop(columns=['feature_sign','position','s','rate'],inplace=True)
    coin[column] = coin[column].shift(-1)
    return coin
coin = find_peak(coin,88,'pos88',0.01)
coin = find_peak(coin,188,'pos188',0.03)
coin = find_peak(coin,288,'pos288',0.05)
coin = find_peak(coin,388,'pos388',0.08)
coin = find_peak(coin,588,'pos388',0.12)

coin['pos'] = coin[['pos88','pos188','pos288','pos388']].sum(axis=1)
coin.drop(columns=['pos88','pos188','pos288','pos388'],inplace=True)
coin.loc[(coin['pos']>=4),'pos_'] = 1 
coin.loc[(coin['pos']<=-4),'pos_'] = -1 
coin['pos'] = coin['pos_']
coin['pos'].fillna(value=0,inplace=True)
coin.drop(columns=['pos_'],inplace=True)

In [14]:
coin_t = coin
coin_p = coin_t.copy()
coin_p['predict'] = coin_p['pos']
trace1 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['close'],
            name = 'price', # Style name/legend entry with html tags
            connectgaps=True
    )
trace2 = go.Scatter(
        x=coin_p[coin_p['predict']==1]['datetime'],
        y=coin_p[coin_p['predict']==1]['close'],
        name = '1',
        mode = 'markers'
    )
trace3 = go.Scatter(
        x=coin_p[coin_p['predict']==-1]['datetime'],
        y=coin_p[coin_p['predict']==-1]['close'],
        name = '2',
        mode = 'markers'
    )

fig = dict(data=[trace1,trace2,trace3])
plotly.offline.plot(fig)

'temp-plot.html'

In [4]:
from pykalman import KalmanFilter
def kalman(data,n_dim_obs=1,in_state_mean=7000,in_state_covar=20,obs_covar=10):
    kf = KalmanFilter(n_dim_obs=n_dim_obs,#观察空间维度
    initial_state_mean=in_state_mean, #初始状态分布的均值
    initial_state_covariance=in_state_covar,#初始状态分布的协方差
    observation_covariance=obs_covar#t+1时刻的观测协方差
    )

    state_means, state_covariance = kf.filter(data['close'])

    return state_means

def k_MA(data,k_length=5,MA_length=10,MA='ma5'):

    k_close = []
    k_list = []
    for i in range(0,len(list1)):
        if i==0:
            k_close.append(list1[0])
        if i%(k_length)==0 and i!=0:
            k_close.append(list1[i])

    k_df = pd.DataFrame(data=k_close, columns=['close'])
    k_df[MA] = k_df['close'].rolling(MA_length).mean()

    for i in range(len(data)):
        if i<=k_length*MA_length:
            k_list.append(data[i])
        if i>k_length*MA_length:
            if i%k_length==0:
                k_list.append(k_df[MA][i//k_length])
            if len(list1) - i > 0 and i%k_length!=0:
                k_list.append(k_list[-1])

    return k_list

def calc_ma(serise,ma): 
    return serise.rolling(ma).mean()
malist = []
volume_list = []
macd_list = []
cmo_list = []
mom_list = []

for ma in [3,5, 7, 9,14, 21, 30, 50,70,90,120,150,180,230,280,360]: 

    coin['ADX'+ str(ma)] = talib.ADX(coin['high'], coin['low'], coin['close'], timeperiod=ma)
    coin['KAMA'+ str(ma)] = talib.KAMA(coin['close'], timeperiod=ma)

    coin.loc[(0 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 10), 'self_ma'+ str(ma)] = coin['close'].rolling(30).mean()
    coin.loc[(10 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 20), 'self_ma'+ str(ma)] = coin['close'].rolling(27).mean()
    coin.loc[(20 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 30), 'self_ma'+ str(ma)] = coin['close'].rolling(23).mean()
    coin.loc[(30 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 40), 'self_ma'+ str(ma)] = coin['close'].rolling(20).mean()
    coin.loc[(40 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 50), 'self_ma'+ str(ma)] = coin['close'].rolling(16).mean()
    coin.loc[(50 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 60), 'self_ma'+ str(ma)] = coin['close'].rolling(12).mean()
    coin.loc[(60 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 70), 'self_ma'+ str(ma)] = coin['close'].rolling(7).mean()
    coin.loc[(70 < coin['ADX'+ str(ma)])&(coin['ADX'+ str(ma)] <= 80), 'self_ma'+ str(ma)] = coin['close'].rolling(5).mean()
    coin.loc[(80 < coin['ADX'+ str(ma)]), 'self_ma'+ str(ma)] = coin['close'].rolling(2).mean()
    



for ma in [3,5, 7, 14, 21, 30, 50,70,90,120,150,180,230,280,360]: 
    coin['coinma'+str(ma)] = calc_ma(coin['close'], ma)
    malist.append('coinma'+str(ma))
    coin['volumema'+str(ma)] = calc_ma(coin['volume'], ma)
    volume_list.append('volumema'+str(ma))
    
    
coin['SAR'] = talib.SAR(coin['high'], coin['low'])

count = len(coin)
coin['kalman'] = kalman(coin,1,3650,20,120)[0:count]

# fit = SimpleExpSmoothing(coin['close']).fit(smoothing_level=0.8,optimized=False)
# coin['holtwinters'] = fit.fittedvalues

# fit2 = Holt(coin['close'], exponential=True).fit(smoothing_level=0.8, smoothing_slope=0.2, optimized=False)
# coin['holtwinters2'] = fit2.fittedvalues

drop_list = list(coin.columns)  
    
for ma in [3,5, 7, 14, 21, 30, 50,70,90,120,150,180,230,280,360]: 
    if ma > 14:
        macd, macdsignal, macdhist = talib.MACD(coin['close'],fastperiod=7, slowperiod=ma, signalperiod=9)
        coin['macdhist'+ str(ma)] = macdhist
        macd_list.append('macdhist'+ str(ma))
    
    coin['CMO'+ str(ma)] = talib.CMO(coin['close'], timeperiod=ma)
    coin['CMO'+ str(ma)] = coin['CMO'+ str(ma)].rolling(5).mean()
    cmo_list.append('CMO'+ str(ma))
    
    coin['MOM'+ str(ma)] = talib.MOM(coin['close'], timeperiod=ma)
    coin['MOM'+ str(ma)] = coin['MOM'+ str(ma)].rolling(5).mean()
    mom_list.append('MOM'+ str(ma))
    
    coin['AROONOSC'+str(ma)] = talib.AROONOSC(coin['high'], coin['low'],timeperiod=ma)
    
    coin['CCI'+str(ma)] = talib.CCI(coin['high'], coin['low'], coin['close'],timeperiod=ma)
#     coin['CCI'+str(ma)] = coin['CCI'+str(ma)].rolling(5).mean()
    coin['DX'+ str(ma)] = talib.DX(coin['high'], coin['low'], coin['close'], timeperiod=ma)
    coin['ADX'+ str(ma)] = talib.ADX(coin['high'], coin['low'], coin['close'], timeperiod=ma)

    coin['ATR_'+ str(ma)] = talib.ATR(coin['high'], coin['low'], coin['close'], timeperiod=ma)
    
    malist.append('self_ma'+ str(ma))
    malist.append('KAMA'+ str(ma))
    
    
  

# macd, macdsignal, macdhist = talib.MACD(coin['close'])
# coin['macdhist'] = macdhist
# coin['CMO'] = talib.CMO(coin['close'], timeperiod=21)
# coin['CMO'] = coin['CMO'].rolling(5).mean()
# coin['MOM'] = talib.MOM(coin['close'], timeperiod=21)
# coin['MOM'] = coin['MOM'].rolling(5).mean()


# drop_list = list(coin.columns)

# for c in cmo_list:
#     coin[c + '_signal'] = 0
#     coin.loc[coin[c] > 20, c + '_signal'] = 1
#     coin.loc[coin[c] < -20, c + '_signal'] = -1

# for c in mom_list:
#     coin[c + '_signal'] = 0
#     coin.loc[coin[c] > 100, c + '_signal'] = 1
#     coin.loc[coin[c] < -100, c + '_signal'] = -1
    
# for c in macd_list:
#     coin.loc[coin[c]>0 , c + '_signal'] = 1
#     coin.loc[coin[c]<=0 , c + '_signal'] = -1

    
    

malist.append('close')
malist.append('high')
malist.append('low')
malist.append('open')
malist.append('SAR')
malist.append('kalman')
# malist.append('holtwinters')
# malist.append('holtwinters2')

volume_list.append('volume')
comclose_list = list(combinations(malist, 2))
comvolume_list = list(combinations(volume_list, 2))
for c in comclose_list:
    coin.loc[coin[c[0]]>coin[c[1]], c[0]+'_'+c[1]] = 1
    coin.loc[coin[c[0]]<=coin[c[1]], c[0]+'_'+c[1]] = -1
for c in comvolume_list:
    coin.loc[coin[c[0]]>coin[c[1]], c[0]+'_'+c[1]] = 1
    coin.loc[coin[c[0]]<=coin[c[1]], c[0]+'_'+c[1]] = -1



In [5]:
columns = list(coin.columns)
for i in drop_list:
    columns.remove(i)
for column in columns:
    coin[column+'_shift2'] = coin[column].shift(2)
    coin[column+'_shift3'] = coin[column].shift(3)
    coin[column+'_shift5'] = coin[column].shift(5)
    coin[column+'_shift7'] = coin[column].shift(7)
    coin[column+'_shift9'] = coin[column].shift(9)
#     coin[column+'_shift12'] = coin[column].shift(12)
#     coin[column+'_shift14'] = coin[column].shift(14)
#     coin[column+'_shift18'] = coin[column].shift(18)
#     coin[column+'_shift21'] = coin[column].shift(21)
#     coin[column+'_shift30'] = coin[column].shift(30)

In [6]:
# drop_list.remove('up')
# drop_list.remove('down')

coin.dropna(inplace=True)
coin_ =coin.drop(columns=drop_list)
coin_train = coin_[:int(len(coin_)*0.8)]
coin_test = coin_[int(len(coin_)*0.8):]
coin_train_x =coin_train
coin_train_y_classification = coin['pos'][:int(len(coin_)*0.8)]
coin_test_x = coin_test
coin_test_y_classification = coin['pos'][int(len(coin_)*0.8):]
X = coin
Y = coin['pos']

In [7]:
coin_train_x = coin_train_x.values
coin_train_y_classification = coin_train_y_classification.values
coin_test_x = coin_test_x.values
coin_test_y_classification = coin_test_y_classification.values

In [25]:
import argparse
import numpy as np
import sys
from keras.datasets import mnist
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")

from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", dest="model", type=str, default=None, help="gcfoest Net Model File")
    args = parser.parse_args()
    return args



def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 1
    ca_config["n_classes"] = 7
    ca_config["estimators"] = []
#     ca_config["estimators"].append(
#             {"n_folds": 5, "type": "XGBClassifier", "n_estimators": 100, "max_depth": 5,
#              "objective": "multi:softprob", "silent": True, "nthread": -1, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 1000, "max_depth": None, "n_jobs": -1})
#     ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 1000, "max_depth": None, "n_jobs": -1})
#     ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

Using TensorFlow backend.


In [26]:
# args = parse_args()
# if args.model is None:
#     config = get_toy_config()
# else:
#     config = load_json(args.model)
    
config = get_toy_config()

gc = GCForest(config)
X_train_enc = gc.fit_transform(coin_train_x, coin_train_y_classification,X_test=coin_test_x,y_test=coin_test_y_classification)

X_groups_train.shape=[(17305, 1446)],y_train.shape=(17305,),X_groups_test.shape=[(4327, 1446)],y_test.shape=(4327,)
group_dims=[1446]
group_starts=[0]
group_ends=[1446]
X_train.shape=(17305, 1446),X_test.shape=(4327, 1446)
[layer=0] look_indexs=[0], X_cur_train.shape=(17305, 1446), X_cur_test.shape=(4327, 1446)
Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=98.96%
Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=99.13%
Accuracy(layer_0 - estimator_0 - 5_folds.train_2.predict)=98.90%
Accuracy(layer_0 - estimator_0 - 5_folds.train_3.predict)=99.13%
Accuracy(layer_0 - estimator_0 - 5_folds.train_4.predict)=99.05%
Accuracy(layer_0 - estimator_0 - 5_folds.train_cv.predict)=99.03%
Accuracy(layer_0 - estimator_0 - 5_folds.test.predict)=43.26%
Accuracy(layer_0 - train.classifier_average)=99.03%
Accuracy(layer_0 - test.classifier_average)=43.26%
[layer=1] look_indexs=[0], X_cur_train.shape=(17305, 1453), X_cur_test.shape=(4327, 1453)
Accuracy(layer_1 - estimator_0 - 5_folds.

In [38]:
predict_y = gc.predict(coin_test_x)

X_groups_test.shape=[(856, 1446)]
group_dims=[1446]
X_test.shape=(856, 1446)
[layer=0] look_indexs=[0], X_cur_test.shape=(856, 1446)
[layer=1] look_indexs=[0], X_cur_test.shape=(856, 1453)


In [74]:
rate_p = gc.predict_proba(coin_test_x)

X_groups_test.shape=[(3851, 979)]
group_dims=[979]
X_test.shape=(3851, 979)
[layer=0] look_indexs=[0], X_cur_test.shape=(3851, 979)
[layer=1] look_indexs=[0], X_cur_test.shape=(3851, 993)


In [39]:
accuracy_score(coin_test_y_classification, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [856, 4327]

In [11]:
coin_t = coin[int(len(coin_)*0.8):]
coin_t['predict'] = predict_y
# coin_t['-1_'] = rate_p.T[2]
# coin_t['0_'] = rate_p.T[0]
# coin_t['1_'] = rate_p.T[1]
coin_p = coin_t.copy()
# coin_p['predict'] = coin_p['pos']
trace1 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['close'],
            name = 'price', # Style name/legend entry with html tags
            connectgaps=True
    )
trace2 = go.Scatter(
        x=coin_p[coin_p['predict']==1]['datetime'],
        y=coin_p[coin_p['predict']==1]['close'],
        name = '1',
        mode = 'markers'
    )
trace3 = go.Scatter(
        x=coin_p[coin_p['predict']==-1]['datetime'],
        y=coin_p[coin_p['predict']==-1]['close'],
        name = '2',
        mode = 'markers'
    )
# trace4 = go.Scatter(
#         x=coin_p[coin_p['predict']==3]['datetime'],
#         y=coin_p[coin_p['predict']==3]['close'],
#         name = '3',
#         mode = 'markers'
#     )
# trace5 = go.Scatter(
#         x=coin_p[coin_p['predict']==4]['datetime'],
#         y=coin_p[coin_p['predict']==4]['close'],
#         name = '4',
#         mode = 'markers'
#     )
# trace6 = go.Scatter(
#         x=coin_p[coin_p['predict']==5]['datetime'],
#         y=coin_p[coin_p['predict']==5]['close'],
#         name = '5',
#         mode = 'markers'
#     )
# trace7 = go.Scatter(
#         x=coin_p[coin_p['predict']==6]['datetime'],
#         y=coin_p[coin_p['predict']==6]['close'],
#         name = '6',
#         mode = 'markers'
#     )
# trace8 = go.Scatter(
#         x=coin_p[coin_p['predict']==0]['datetime'],
#         y=coin_p[coin_p['predict']==0]['close'],
#         name = '0',
#         mode = 'markers'
#     )
# trace9 = go.Scatter(
#         x=coin_p[coin_p['predict']==-1]['datetime'],
#         y=coin_p[coin_p['predict']==-1]['close'],
#         name = '-1',
#         mode = 'markers'
#     )

# trace5 = go.Scatter(
#             x=coin_p['datetime'],
#             y=coin_p['ATR'],
#             name = 'atr', # Style name/legend entry with html tags
#             connectgaps=True
#     )

fig = dict(data=[trace1,trace2,trace3])
plotly.offline.plot(fig)

'temp-plot.html'

In [12]:

coin_p = coin_t.copy()


# coin_p['-1_'] = coin_p['-1_'].rolling(30).mean()
# coin_p['1_'] = coin_p['1_'].rolling(30).mean()
# coin_p['0_'] = coin_p['0_'].rolling(30).mean()
# coin_p['predict'] = 0
# coin_p.loc[(coin_p['-1_']>coin_p['1_'])&(coin_p['-1_']>0), 'predict'] = -1
# coin_p.loc[(coin_p['1_']>coin_p['-1_'])&(coin_p['1_']>0), 'predict'] = 1


# coin_p.loc[coin_p['predict']==0,'predict'] = -1
# coin_p.loc[(coin_p['predict']>0)&(coin_p['predict']<6),'predict']=0
# coin_p.loc[coin_p['predict']==6,'predict'] = 1

# coin_p.loc[coin_p['predict']<3,'predict'] = -1
# coin_p.loc[coin_p['predict']>3,'predict'] = 1


# coin_p.loc[coin_p['predict']==2,'predict'] = -1

coin_p.reset_index(drop=True,inplace=True)

# coin['pos'] = 0
# coin.loc[(coin['predict']==1)&(coin['close']>coin['SAR']),'pos'] = 1
# coin.loc[(coin['predict']==-1)&(coin['close']<coin['SAR']),'pos'] = -1

# coin['position'] = coin['pos'].shift()

# coin['position'] = coin['position'].shift()
coin_p['position'] = coin_p['predict'].shift()

# coin.loc[coin['position']==0,'position'] = np.nan
# coin['position'].fillna(method='ffill',inplace=True)
# coin.loc[(coin['position']==-1),'position'] = 0
coin_p.loc[(coin_p['position']!=1)&(coin_p['position'].shift(-1)==1),'signal'] = 1
coin_p.loc[(coin_p['position']!=-1)&(coin_p['position'].shift(-1)==-1),'signal'] = -1
coin_p.loc[(coin_p['position']!=0)&(coin_p['position'].shift(-1)==0),'signal'] = 0
coin_p['rate'] = (coin_p['close']- coin_p['close'].shift())/coin_p['close'].shift()
coin_p['chg_m'] = coin_p['rate'] * coin_p['position'] * 1
coin_p['stock_curve'] = coin_p['close']/coin_p.at[0,'open']
coin_p['curve'] = (1+coin_p['chg_m'])
coin_p.loc[(coin_p['signal']==1)|(coin_p['signal']==-1)|(coin_p['signal']==0),'curve'] = coin_p['curve'] -0.0007*3
coin_p['curve'] = coin_p['curve'].cumprod()

curve_max = []
c = 1 
for i in coin_p['curve']:
    if i > c:
        curve_max.append(i)
        c = i
    else:
        curve_max.append(c)
coin_p['curve_max'] = curve_max
coin_p['huice'] = 1 - (coin_p['curve_max'] - coin_p['curve']) / coin_p['curve_max']

In [13]:
subplots = tools.make_subplots(2,1,shared_xaxes=True)
trace = go.Candlestick(x=coin_p['datetime'],
                       open=coin_p['open'],
                       high=coin_p['high'],
                       low=coin_p['low'],
                       close=coin_p['close'])

trace1 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['close'],
            name = 'price',
            connectgaps=True
        )

signal1 = go.Scatter(
        x=coin_p[coin_p['signal'] == 1]['datetime'],
        y=coin_p[coin_p['signal'] == 1]['close'],
        name = 'signal_buy',
        connectgaps=True,
        mode = 'markers'
    )
signal2 = go.Scatter(
        x=coin_p[coin_p['signal'] == -1]['datetime'],
        y=coin_p[coin_p['signal'] == -1]['close'],
        name = 'signal_sell',
        connectgaps=True,
        mode = 'markers'
    )
signal3 = go.Scatter(
        x=coin_p[coin_p['signal'] == 0]['datetime'],
        y=coin_p[coin_p['signal'] == 0]['close'],
        name = 'ping',
        connectgaps=True,
        mode = 'markers'
    )

trace2 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['curve'],
            name = 'curve',
            connectgaps=True
        )

# trace3 = go.Scatter(
#             x=coin['datetime'],
#             y=coin['huice'],
#             name = 'huice',
#             connectgaps=True
#         )

subplots.append_trace(trace1,1,1)
subplots.append_trace(signal1,1,1)
subplots.append_trace(signal2,1,1)
subplots.append_trace(signal3,1,1)
subplots.append_trace(trace2,2,1)
# subplots.append_trace(trace3,3,1)
plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



'temp-plot.html'

4269

In [8]:
estimator = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=8888, n_jobs=-1,
            oob_score=False, random_state=23, verbose=0,
            warm_start=False)
estimator.fit(coin_train_x,coin_train_y_classification)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=8888, n_jobs=-1,
            oob_score=False, random_state=23, verbose=0, warm_start=False)

In [9]:
predict_y = estimator.predict(coin_test_x)
coin_t = estimator.predict_proba(coin_test_x)

In [10]:
accuracy_score(coin_test_y_classification, predict_y)

0.5832910106653123

In [7]:
param_grid = {'min_samples_leaf':[1,23,60,90,120,180,250,360,400,600,900,1000,1200,1300,1500]}
estimator = RandomForestClassifier(n_jobs=-1,n_estimators=2000)
grid = GridSearchCV(estimator, param_grid, cv=10, scoring='accuracy')
grid.fit(coin_, coin['pos'])

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_leaf': [1, 23, 60, 90, 120, 180, 250, 360, 400, 600, 900, 1000, 1200, 1300, 1500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [8]:
grid.grid_scores_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [11]:
grid.score

<bound method BaseSearchCV.score of GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_leaf': [1, 23, 60, 90, 120, 180, 250, 360, 400, 600, 900, 1000, 1200, 1300, 1500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)>

In [9]:
grid.best_params_

{'min_samples_leaf': 1500}

In [ ]:
coin.dropna(inplace=True)
coin_test_x =coin.drop(columns=drop_list)
predict_y = estimator.predict(coin_test_x)
p = estimator.predict_proba(coin_test_x)
coin['-1_'] = p.T[0]
coin['0_'] = p.T[1]
coin['1_'] = p.T[2]
coin['predict'] = predict_y
coin_t = coin

In [14]:
with open('BTCUSDT_vkline_1212.pickle', 'wb') as f:
    pickle.dump(estimator, f)

In [ ]:
with open('BTCUSDT_vkline_1212.pickle', 'rb') as f:
    estimator = pickle.load(f)

In [22]:
coin_t = coin[int(len(coin_)*0.8):]
coin_t['predict'] = predict_y
coin_t['-1_'] = p.T[0]
coin_t['0_'] = p.T[1]
coin_t['1_'] = p.T[2]

In [8]:
coin_p = coin.copy()
# coin_p['predict'] = coin_p['pos']
trace1 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['close'],
            name = 'price', # Style name/legend entry with html tags
            connectgaps=True
    )
trace2 = go.Scatter(
        x=coin_p[coin_p['predict']==1]['datetime'],
        y=coin_p[coin_p['predict']==1]['close'],
        name = 'predict_buy',
        mode = 'markers'
    )
trace3 = go.Scatter(
        x=coin_p[coin_p['predict']==-1]['datetime'],
        y=coin_p[coin_p['predict']==-1]['close'],
        name = 'predict_sell',
        mode = 'markers'
    )
# trace5 = go.Scatter(
#             x=coin_p['datetime'],
#             y=coin_p['ATR'],
#             name = 'atr', # Style name/legend entry with html tags
#             connectgaps=True
#     )

fig = dict(data=[trace1,trace2,trace3])
plotly.offline.plot(fig)

KeyError: 'predict'

In [103]:
subplots = tools.make_subplots(1,1,shared_xaxes=True)
trace1 = go.Scatter(
            x=coin['datetime'],
            y=coin['close'],
            name = 'close',
            connectgaps=True
        )

trace2 = go.Scatter(
            x=coin[coin['pos'] == 1]['datetime'],
            y=coin[coin['pos'] == 1]['close'],
            name = 'buy',
            connectgaps=True,
            mode='markers'
    
        )
trace3 = go.Scatter(
            x=coin[coin['pos'] == -1]['datetime'],
            y=coin[coin['pos'] == -1]['close'],
            name = 'sell',
            connectgaps=True,
            mode='markers'
    
        )

subplots.append_trace(trace1,1,1)
subplots.append_trace(trace2,1,1)
subplots.append_trace(trace3,1,1)
plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



'temp-plot.html'

In [23]:
coin_p = coin_t.copy()

# coin_p['predict'] = 0
# coin_p.loc[coin_p['-1_']>0.6, 'predict'] = -1
# coin_p.loc[coin_p['1_']>0.6, 'predict'] = 1

subplots = tools.make_subplots(2,1,shared_xaxes=True)
trace1 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['close'],
            name = 'close',data
            connectgaps=True
        )
trace_buy = go.Scatter(
        x=coin_p[coin_p['predict']==1]['datetime'],
        y=coin_p[coin_p['predict']==1]['close'],
        name = 'predict_buy',
        mode = 'markers'
    )
trace_sell = go.Scatter(
        x=coin_p[coin_p['predict']==-1]['datetime'],
        y=coin_p[coin_p['predict']==-1]['close'],
        name = 'predict_sell',
        mode = 'markers'
    )

trace2 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['-1_'],
            name = '-1_',
            connectgaps=True
        )
trace3 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['0_'],
            name = '0_',
            connectgaps=True
        )
trace4 = go.Scatter(
            x=coin_p['datetime'],
            y=coin_p['1_'],
            name = '1_',
            connectgaps=True
        )

subplots.append_trace(trace1,1,1)
subplots.append_trace(trace_buy,1,1)
subplots.append_trace(trace_sell,1,1)
subplots.append_trace(trace2,2,1)
subplots.append_trace(trace3,2,1)
subplots.append_trace(trace4,2,1)
plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



'temp-plot.html'

In [12]:
coin = coin_t.copy()


# coin['-1_'] = coin['-1_'].rolling(7).mean()
# coin['1_'] = coin['1_'].rolling(7).mean()
# coin['0_'] = coin['0_'].rolling(7).mean()
# coin['predict'] = 0
# coin.loc[(coin['-1_']>coin['1_'])&(coin['-1_']>0), 'predict'] = -1
# coin.loc[(coin['1_']>coin['-1_'])&(coin['1_']>0), 'predict'] = 1


coin.reset_index(drop=True,inplace=True)

# coin['pos'] = 0
# coin.loc[(coin['predict']==1)&(coin['close']>coin['SAR']),'pos'] = 1
# coin.loc[(coin['predict']==-1)&(coin['close']<coin['SAR']),'pos'] = -1

# coin['position'] = coin['pos'].shift()

# coin['position'] = coin['position'].shift()
coin['position'] = coin['predict'].shift()

# coin.loc[coin['position']==0,'position'] = np.nan
# coin['position'].fillna(method='ffill',inplace=True)
# coin.loc[(coin['position']==-1),'position'] = 0
coin.loc[(coin['position']!=1)&(coin['position'].shift(-1)==1),'signal'] = 1
coin.loc[(coin['position']!=-1)&(coin['position'].shift(-1)==-1),'signal'] = -1
coin.loc[(coin['position']!=0)&(coin['position'].shift(-1)==0),'signal'] = 0
coin['rate'] = (coin['close']- coin['close'].shift())/coin['close'].shift()
coin['chg_m'] = coin['rate'] * coin['position'] * 1
coin['stock_curve'] = coin['close']/coin.at[0,'open']
coin['curve'] = (1+coin['chg_m'])
coin.loc[(coin['signal']==1)|(coin['signal']==-1)|(coin['signal']==0),'curve'] = coin['curve'] -0.0007*3
coin['curve'] = coin['curve'].cumprod()

curve_max = []
c = 1 
for i in coin['curve']:
    if i > c:
        curve_max.append(i)
        c = i
    else:
        curve_max.append(c)
coin['curve_max'] = curve_max
coin['huice'] = 1 - (coin['curve_max'] - coin['curve']) / coin['curve_max']

In [13]:
subplots = tools.make_subplots(2,1,shared_xaxes=True)
trace = go.Candlestick(x=coin['datetime'],
                       open=coin['open'],
                       high=coin['high'],
                       low=coin['low'],
                       close=coin['close'])

trace1 = go.Scatter(
            x=coin['datetime'],
            y=coin['close'],
            name = 'price',
            connectgaps=True
        )

signal1 = go.Scatter(
        x=coin[coin['signal'] == 1]['datetime'],
        y=coin[coin['signal'] == 1]['close'],
        name = 'signal_buy',
        connectgaps=True,
        mode = 'markers'
    )
signal2 = go.Scatter(
        x=coin[coin['signal'] == 2]['datetime'],
        y=coin[coin['signal'] == 2]['close'],
        name = 'signal_sell',
        connectgaps=True,
        mode = 'markers'
    )
signal3 = go.Scatter(
        x=coin[coin['signal'] == 0]['datetime'],
        y=coin[coin['signal'] == 0]['close'],
        name = 'ping',
        connectgaps=True,
        mode = 'markers'
    )

trace2 = go.Scatter(
            x=coin['datetime'],
            y=coin['curve'],
            name = 'curve',
            connectgaps=True
        )

# trace3 = go.Scatter(
#             x=coin['datetime'],
#             y=coin['huice'],
#             name = 'huice',
#             connectgaps=True
#         )

subplots.append_trace(trace1,1,1)
subplots.append_trace(signal1,1,1)
subplots.append_trace(signal2,1,1)
subplots.append_trace(signal3,1,1)
subplots.append_trace(trace2,2,1)
# subplots.append_trace(trace3,3,1)
plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



'temp-plot.html'

In [17]:
coin.dropna(inplace=True)
coin_ =coin.drop(columns=drop_list)

train = []
test = [[0,0.2],[0.2,0.4],[0.4,0.6],[0.6,0.8],[0.8,1]]
for i in test:
    coin_train = coin_[:int(len(coin_)*i[0])].append(coin_[int(len(coin_)*i[1]):])
    coin_test = coin_[int(len(coin_)*i[0]):int(len(coin_)*i[1])]
    coin_train_x =coin_train
    coin_train_y_classification = coin['pos'][:int(len(coin_)*i[0])].append(coin['pos'][int(len(coin_)*i[1]):])
    coin_test_x = coin_test
    coin_test_y_classification = coin['pos'][int(len(coin_)*0.8):]
    X = coin
    Y = coin['pos']
    
    estimator = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=6000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
    estimator.fit(coin_train_x,coin_train_y_classification)
    predict_y = estimator.predict(coin_test_x)
#     p = estimator.predict_proba(coin_test_x)
    
    
    coin_t = coin[int(len(coin_)*i[0]):int(len(coin_)*i[1])]
    
#     coin_t['predict'] = predict_y
    
#     coin_t['-1_'] = p.T[0]
#     coin_t['0_'] = p.T[1]
#     coin_t['1_'] = p.T[2]

    coin_t['predict'] = predict_y
    coin_p = coin_t.copy()


#     coin_p.loc[coin_p['predict']<3,'predict'] = -1
#     coin_p.loc[coin_p['predict']>3,'predict'] = 1

    coin_p.reset_index(drop=True,inplace=True)

    coin_p['position'] = coin_p['predict'].shift()

    coin_p.loc[(coin_p['position']!=1)&(coin_p['position'].shift(-1)==1),'signal'] = 1
    coin_p.loc[(coin_p['position']!=-1)&(coin_p['position'].shift(-1)==-1),'signal'] = -1
    coin_p.loc[(coin_p['position']!=0)&(coin_p['position'].shift(-1)==0),'signal'] = 0
    coin_p['rate'] = (coin_p['close']- coin_p['close'].shift())/coin_p['close'].shift()
    coin_p['chg_m'] = coin_p['rate'] * coin_p['position'] * 1
    coin_p['stock_curve'] = coin_p['close']/coin_p.at[0,'open']
    coin_p['curve'] = (1+coin_p['chg_m'])
    coin_p.loc[(coin_p['signal']==1)|(coin_p['signal']==-1)|(coin_p['signal']==0),'curve'] = coin_p['curve'] -0.0007*3
    coin_p['curve'] = coin_p['curve'].cumprod()

    curve_max = []
    c = 1 
    for i in coin_p['curve']:
        if i > c:
            curve_max.append(i)
            c = i
        else:
            curve_max.append(c)
    coin_p['curve_max'] = curve_max
    coin_p['huice'] = 1 - (coin_p['curve_max'] - coin_p['curve']) / coin_p['curve_max']
    
    subplots = tools.make_subplots(2,1,shared_xaxes=True)
    trace = go.Candlestick(x=coin_p['datetime'],
                           open=coin_p['open'],
                           high=coin_p['high'],
                           low=coin_p['low'],
                           close=coin_p['close'])

    trace1 = go.Scatter(
                x=coin_p['datetime'],
                y=coin_p['close'],
                name = 'price',
                connectgaps=True
            )

    signal1 = go.Scatter(
            x=coin_p[coin_p['signal'] == 1]['datetime'],
            y=coin_p[coin_p['signal'] == 1]['close'],
            name = 'signal_buy',
            connectgaps=True,
            mode = 'markers'
        )
    signal2 = go.Scatter(
            x=coin_p[coin_p['signal'] == -1]['datetime'],
            y=coin_p[coin_p['signal'] == -1]['close'],
            name = 'signal_sell',
            connectgaps=True,
            mode = 'markers'
        )
    signal3 = go.Scatter(
            x=coin_p[coin_p['signal'] == 0]['datetime'],
            y=coin_p[coin_p['signal'] == 0]['close'],
            name = 'ping',
            connectgaps=True,
            mode = 'markers'
        )

    trace2 = go.Scatter(
                x=coin_p['datetime'],
                y=coin_p['curve'],
                name = 'curve',
                connectgaps=True
            )

    # trace3 = go.Scatter(
    #             x=coin['datetime'],
    #             y=coin['huice'],
    #             name = 'huice',
    #             connectgaps=True
    #         )

    subplots.append_trace(trace1,1,1)
    subplots.append_trace(signal1,1,1)
    subplots.append_trace(signal2,1,1)
    subplots.append_trace(signal3,1,1)
    subplots.append_trace(trace2,2,1)
    # subplots.append_trace(trace3,3,1)
    plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



In [166]:
coin.dropna(inplace=True)
coin_ =coin.drop(columns=drop_list)

train = [[0,0.2],[0.2,0.4],[0.4,0.6],[0.6,0.8]]
test = [[0.2,0.4],[0.4,0.6],[0.6,0.8],[0.8,1]]
for i in train:
    coin_train = coin_[int(len(coin_)*i[0]):int(len(coin_)*i[1])]
    coin_test = coin_[int(len(coin_)*(i[0]+0.2)):int(len(coin_)*(i[1]+0.1))]
    coin_train_x =coin_train
    coin_train_y_classification = coin['pos'][int(len(coin_)*i[0]):int(len(coin_)*i[1])]
    coin_test_x = coin_test
    coin_test_y_classification = coin['pos'][int(len(coin_)*0.8):]
    X = coin
    Y = coin['pos']
    
    estimator = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
    estimator.fit(coin_train_x,coin_train_y_classification)
#     predict_y = estimator.predict(coin_test_x)
    p = estimator.predict_proba(coin_test_x)
    
    
    coin_t = coin[int(len(coin_)*(i[0]+0.2)):int(len(coin_)*(i[1]+0.1))]
    
#     coin_t['predict'] = predict_y
    
    coin_t['-1_'] = p.T[0]
    coin_t['0_'] = p.T[1]
    coin_t['1_'] = p.T[2]

    
    coin_p = coin_t.copy()


    coin_p['-1_'] = coin_p['-1_'].rolling(7).mean()
    coin_p['1_'] = coin_p['1_'].rolling(7).mean()
    coin_p['0_'] = coin_p['0_'].rolling(7).mean()
    coin_p['predict'] = 0
    coin_p.loc[(coin_p['-1_']>coin_p['1_']), 'predict'] = -1
    coin_p.loc[(coin_p['1_']>coin_p['-1_']), 'predict'] = 1

    coin_p.reset_index(drop=True,inplace=True)

    coin_p['position'] = coin_p['predict'].shift()

    coin_p.loc[(coin_p['position']!=1)&(coin_p['position'].shift(-1)==1),'signal'] = 1
    coin_p.loc[(coin_p['position']!=-1)&(coin_p['position'].shift(-1)==-1),'signal'] = -1
    coin_p.loc[(coin_p['position']!=0)&(coin_p['position'].shift(-1)==0),'signal'] = 0
    coin_p['rate'] = (coin_p['close']- coin_p['close'].shift())/coin_p['close'].shift()
    coin_p['chg_m'] = coin_p['rate'] * coin_p['position'] * 3
    coin_p['stock_curve'] = coin_p['close']/coin_p.at[0,'open']
    coin_p['curve'] = (1+coin_p['chg_m'])
    coin_p.loc[(coin_p['signal']==1)|(coin_p['signal']==-1)|(coin_p['signal']==0),'curve'] = coin_p['curve'] -0.0007*5
    coin_p['curve'] = coin_p['curve'].cumprod()

    curve_max = []
    c = 1 
    for i in coin_p['curve']:
        if i > c:
            curve_max.append(i)
            c = i
        else:
            curve_max.append(c)
    coin_p['curve_max'] = curve_max
    coin_p['huice'] = 1 - (coin_p['curve_max'] - coin_p['curve']) / coin_p['curve_max']
    
    subplots = tools.make_subplots(2,1,shared_xaxes=True)
    trace = go.Candlestick(x=coin_p['datetime'],
                           open=coin_p['open'],
                           high=coin_p['high'],
                           low=coin_p['low'],
                           close=coin_p['close'])

    trace1 = go.Scatter(
                x=coin_p['datetime'],
                y=coin_p['close'],
                name = 'price',
                connectgaps=True
            )

    signal1 = go.Scatter(
            x=coin_p[coin_p['signal'] == 1]['datetime'],
            y=coin_p[coin_p['signal'] == 1]['close'],
            name = 'signal_buy',
            connectgaps=True,
            mode = 'markers'
        )
    signal2 = go.Scatter(
            x=coin_p[coin_p['signal'] == -1]['datetime'],
            y=coin_p[coin_p['signal'] == -1]['close'],
            name = 'signal_sell',
            connectgaps=True,
            mode = 'markers'
        )
    signal3 = go.Scatter(
            x=coin_p[coin_p['signal'] == 0]['datetime'],
            y=coin_p[coin_p['signal'] == 0]['close'],
            name = 'ping',
            connectgaps=True,
            mode = 'markers'
        )

    trace2 = go.Scatter(
                x=coin_p['datetime'],
                y=coin_p['curve'],
                name = 'curve',
                connectgaps=True
            )

    # trace3 = go.Scatter(
    #             x=coin['datetime'],
    #             y=coin['huice'],
    #             name = 'huice',
    #             connectgaps=True
    #         )

    subplots.append_trace(trace1,1,1)
    subplots.append_trace(signal1,1,1)
    subplots.append_trace(signal2,1,1)
    subplots.append_trace(signal3,1,1)
    subplots.append_trace(trace2,2,1)
    # subplots.append_trace(trace3,3,1)
    plotly.offline.plot(subplots)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



In [58]:
import asyncio  
import time
import requests

def get_d():
    print(1111111)
#     loop = asyncio.get_event_loop()
#     await loop.run_in_executor(None, requests.get, 'http://www.google.com')
    print(requests.get('http://www.google.com'))

async def fun():  
    print('hello word')
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, get_d, )
#     await get_d()
    print('hahah')
    return 'miao'  

async def fun2():  
    print('hello word2')  
    return 'miao' 
   
loop = asyncio.get_event_loop()  
task1 = loop.create_task(fun())  
task2 = loop.create_task(fun2())  

tasks = [task2,task1]
for task in tasks:
#     print(task)
    loop.run_until_complete(task)  
# loop.run_until_complete(task2) 

hello word
1111111
hello word2
<Response [200]>
hahah
222222222222


In [31]:
new_loop = asyncio.new_event_loop()
asyncio.run_coroutine_threadsafe(fun(), new_loop)
asyncio.run_coroutine_threadsafe(fun2(), new_loop)


<Future at 0x1b9259489b0 state=pending>

In [32]:
new_loop.call_soon_threadsafe(fun, )

<Handle fun() at <ipython-input-30-bdaa72663b0e>:10>

In [ ]:
task1.